In [17]:
import pandas as pd
import os

In [18]:
# load all sheets from drive
# sheet ID comes from URL
sheet_ids = [
            '1Pxs2JngjAQ_iqTtVuzy0xGcyPP7K7F-rumOiN3DcQJI', 
            '1Ktl2TXZPVme2NgTTt-_MivE7X-sTRnrZYHNj6LTzb9I', 
            '1y5Zuh-v-LyRmxurUzpgcfHHdh2eiybRgGiD5K6bYLT8', 
            '1teYR0T8bsXBTTcb1cf00UNFXDgXzmOCh-W2dl7I_fFY',
            '1p0ErgEP94Y-GhG9K550wsTU6E0SNP8aaTNa0rtKD9pY'
            ] 
sheet_names = [
                'Curso_Basico_Sentence_Pairs', 
                'PS_jw_website', 
                'Warao-Aribu-Learning-Guide-Data',
                'Trab_Warao_Completo',
                'Bible_verse_pairs'
                ]

df_lst = [pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()]

for i in range(len(sheet_ids)):
    sheet_id = sheet_ids[i]
    sheet_name = sheet_names[i]
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

    df_lst[i] = pd.read_csv(url)

In [19]:
df_curso_basico, df_ps_jw_website, df_warao_aribu_guide, df_trab_warao_completo, df_bible_verses = df_lst[0], df_lst[1], df_lst[2], df_lst[3], df_lst[4]

## Display Data

In [20]:
print(f"Number of examples from {sheet_names[0]}: {df_curso_basico.shape[0]}")
display(df_curso_basico.head(10))

Number of examples from Curso_Basico_Sentence_Pairs: 583


,warao_sentence,spanish_sentence_1,spanish_sentence_2,spanish_sentence_3,spanish_sentence_4
0,¿Waraotuma sina?,¿Quienes son los warao?,NaN,NaN,NaN
1,Ka ribu isía,Sobre nuestro idioma warao.,NaN,NaN,NaN
2,Ine buratana najoroya,Yo como cambur,NaN,NaN,NaN
3,Iji buratana najoroya,Tú comes cambur,NaN,NaN,NaN
4,Tai buratana najoroya,Ella come cambur,Él come cambur,NaN,NaN
5,Oko Buratana najoroya,Nosotros comemos cambur,Nosotras comemos cambur,NaN,NaN
6,Yatu buratana najoroya,Ustedes comen cambur,NaN,NaN,NaN
7,Tatuma buratana najoroya,Ellos comen cambur,ellas comen cambur,NaN,NaN
8,Ine karata teribuya,Yo estudio,NaN,NaN,NaN
9,Ine Jomakaba najoroya,Yo como pescado,NaN,NaN,NaN


In [21]:
print(f"Number of examples from {sheet_names[1]}: {df_ps_jw_website.shape[0]}")
display(df_ps_jw_website.head(10))

Number of examples from PS_jw_website: 49


,warao_sentence,spanish_sentence
0,BIBLIA ISIA INAMINAKITANE SABA,PROGRAMA DE CLASES PARTICULARES DE LA BIBLIA
1,Iji Biblia isia naminakore tamaja denokotuma i...,En este curso interactivo de la Biblia descubr...
2,Katukane ine oriwakane jakuna?,¿Cómo puedo tener una vida feliz?
3,Ya jisaka eku orikuba sanamata tane ekorate?,¿Se acabarán algún día la maldad y el sufrimie...
4,Maoriwarao muanaubuae jakutai atae mitera?,¿Volveré a ver a mis seres queridos que han mu...
5,Dio maisia obonobuyara?,¿Se preocupa Dios por mí?
6,"Katukane ine Dio isiko dibakitane ja, tai mari...",¿Cómo tengo que orarle a Dios para que me escu...
7,Naminau katukane oko inaminaya,Conozca nuestro curso
8,Inamina kokotuka amoara omi,El curso completo es gratis.
9,"Karata awai Iji oriwakane ubakuna, ji moate ar...","Incluye el manual Disfrute de la vida, y una B..."


In [22]:
print(f"Number of examples from {sheet_names[2]}: {df_warao_aribu_guide.shape[0]}")
display(df_warao_aribu_guide.head(10))

Number of examples from Warao-Aribu-Learning-Guide-Data: 203


,warao_sentence,spanish_sentence
0,Ka Nonatu saba.,"A Dios, nuestro creador."
1,Ka rima tuma ama ka rani tuma rakate.,A nuestros padres.
2,Nobotomo Delta Amacuro arao kokotuka saba.,A los niños Warao de todas las Comunidades Ind...
3,Nobotomo ainaminamo tuma saba rakate tamaja ka...,Al personal Docente que labora en las diversas...
4,Waraotuma ka sanetamo saba rakate.,A todas aquellas personas que nos ayudaron en ...
5,Warao tuma yori,Gloria al bravo pueblo
6,¡kuare asaya!,Que el yugo lanzó
7,aidamo aribu,La ley respetando
8,nome nokobuae,La virtud y honor
9,¡mojoju tanaka!,¡Abajo cadenas!


In [23]:
print(f"Number of examples from {sheet_names[3]}: {df_trab_warao_completo.shape[0]}")
display(df_trab_warao_completo.head(10))

Number of examples from Trab_Warao_Completo: 129


,warao_sentence,spanish_sentence,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,Tamaja karata ine dioso saba jabatae. Tai naru...,Dedico esta guía a Dios todo poderoso por darm...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Ma rima, Dani , daka Tuma, dajia Tuma Marisa k...","A mis padres, hermanos, mis amigos por enseñar...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Ma inaminatuyakera ma isiko, ma nokoae sina in...",A mi profesor por la paciencia sostenida duran...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Yakera masaba ma inaminanoko. Ma saba moae yak...,Agradezco a la universidad bolivariana de Vene...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ma inamena yakeraja ma konaruae inaminakitane,A mi profesor por llevarnos por el camino corr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Yakeraja m asaba warao kokotuka ma isiko tanae,A todas aquellas personas que me brindaron su ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Inaminakitane eku kabiate nunae tae nunakitane...,Crear cambios en las educaciones es una obliga...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,¿Katukete?,¿Cómo estás?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Bajuka sabuka,Regular,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Yakera yana asida,Estoy mal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
print(f"Number of examples from {sheet_names[4]}: {df_bible_verses.shape[0]}")
display(df_bible_verses.head(10))

Number of examples from Bible_verse_pairs: 7958


,warao_sentence,spanish_sentence
0,Tamaja Jesukirito anobotuma awai. Jesukirito j...,Libro de la genealogía (los antepasados) de Je...
1,Abraham jakutai Isaac arima. Isaac Jacob arima...,"Abraham fue padre de Isaac, Isaac de Jacob, y ..."
2,"Judá jakutai Fares, Zafa tane arima. Tatuma ar...","Judá fue padre de Fares y de Zara, cuya madre ..."
3,Aram Aminadab arima. Aminadab Naasón arima. Na...,"Aram fue padre de Aminadab, Aminadab de Naasón..."
4,Salmón Booz arima. Aranima awai Rahab. Booz Ob...,"Salmón fue padre de Booz, cuya madre fue Rahab..."
5,Isaí airamo David arima. Airamo David Salomón ...,Isaí fue padre del rey David. Y David fue padr...
6,Salomón jakutai Roboam arima. Roboam Abias ari...,"Salomón fue padre de Roboam, Roboam de Abías, ..."
7,Asa Josafat arima. Josafat Joram arima. Joram ...,"Asa fue padre de Josafat, Josafat de Joram, y ..."
8,Uzias Jotam arima. Jotam Acaz arima. Acaz Ezeq...,"Uzías fue padre de Jotam, Jotam de Acaz, y Aca..."
9,Ezequias Manasés arima. Manasés Amón arima. Am...,"Ezequías fue padre de Manasés, Manasés de Amón..."


## Clean up data

### df_curso_basico

In [25]:
# combine alternate translations into a single column
expanded_rows = []

for idx, row in df_curso_basico.iterrows():
    # add original row with spanish_sentence_1
    if pd.notna(row['spanish_sentence_1']):
        expanded_rows.append({
            'warao_sentence': row['warao_sentence'],
            'spanish_sentence': row['spanish_sentence_1']
        })
    
    # check for alternate translations in columns spanish_sentence_2-4
    for col in ['spanish_sentence_2', 'spanish_sentence_3', 'spanish_sentence_4']:
        if pd.notna(row[col]):
            expanded_rows.append({
                'warao_sentence': row['warao_sentence'],
                'spanish_sentence': row[col]
            })

# new dataframe
df_curso_basico_expanded = pd.DataFrame(expanded_rows)

print(f"Original number of examples: {len(df_curso_basico)}")
print(f"Expanded number of examples: {len(df_curso_basico_expanded)}")
display(df_curso_basico_expanded.head(20))


Original number of examples: 583
Expanded number of examples: 838


,warao_sentence,spanish_sentence
0,¿Waraotuma sina?,¿Quienes son los warao?
1,Ka ribu isía,Sobre nuestro idioma warao.
2,Ine buratana najoroya,Yo como cambur
3,Iji buratana najoroya,Tú comes cambur
4,Tai buratana najoroya,Ella come cambur
5,Tai buratana najoroya,Él come cambur
6,Oko Buratana najoroya,Nosotros comemos cambur
7,Oko Buratana najoroya,Nosotras comemos cambur
8,Yatu buratana najoroya,Ustedes comen cambur
9,Tatuma buratana najoroya,Ellos comen cambur


### df_warao_aribu

In [26]:
# if isupper == True, change warao_sentence to lowercase with the first letter capitalized
# Only apply transformation to rows where the string is fully uppercase
mask = df_warao_aribu_guide['warao_sentence'].str.isupper().fillna(False)
df_warao_aribu_guide.loc[mask, 'warao_sentence'] = df_warao_aribu_guide.loc[mask, 'warao_sentence'].str.lower().str.capitalize()

display(df_warao_aribu_guide[2:3])
display(df_warao_aribu_guide[83:94])

,warao_sentence,spanish_sentence
2,Nobotomo Delta Amacuro arao kokotuka saba.,A los niños Warao de todas las Comunidades Ind...


,warao_sentence,spanish_sentence
83,Diana ine naria,ya me voy
84,Diana oko naria,ya nos vamos
85,Diana ji omi,"ya, sin ti"
86,Diana yatu omi,"ya, sin ustedes"
87,Jaketae,hasta mañana
88,"Naru, diana",vete ya
89,"Narukotu, diana",váyanse ya
90,Diana naru ka omi,"Váyanse ya, sin nosotros"
91,amaja ma bitu,aquello es mío
92,amate wajibaka jakotai ka bitu,aquella curiara que esta allá es nuestra


### df_trab_warao_completo

In [27]:
# change certain text that is in all caps text to lowercase with the first letter capitalized
df_trab_warao_completo['warao_sentence'] = df_trab_warao_completo['warao_sentence'].str.lower().str.capitalize()
df_trab_warao_completo = df_trab_warao_completo[['spanish_sentence', 'warao_sentence']]

display(df_trab_warao_completo[100:108])


,spanish_sentence,warao_sentence
100,Las niñas tienen que ir a la escuela,Nobotomo inaminanoko narukitane ja
101,Los maestros tienen que hacer comida,Inamina najoro nanakitane ja
102,Hay que tomar bastante agua,Jo era jobikitane ja
103,Hay que ver bien,Yakera mikitane ja
104,¿Qué haces?,Sina iji tia?
105,"Nada, sentado aquí","Tana, ine dujuya"
106,Vamos a pescar,Jaisa yabakitane
107,Tienes anzuelos para pescar,Iji usibakai ja yabakitane


In [28]:
duplicates_curso_basico = df_curso_basico_expanded[df_curso_basico_expanded.duplicated(keep=False)]
duplicates_warao_aribu = df_warao_aribu_guide[df_warao_aribu_guide.duplicated(keep=False)]
duplicates_ps_jw = df_ps_jw_website[df_ps_jw_website.duplicated(keep=False)]
duplicates_trab = df_trab_warao_completo[df_trab_warao_completo.duplicated(keep=False)]
duplicates_bible = df_bible_verses[df_bible_verses.duplicated(keep=False)]

print(f"Number of duplicate examples in df_curso_basico_expanded: {len(duplicates_curso_basico)}")
print(f"Number of duplicate examples in df_warao_aribu_guide: {len(duplicates_warao_aribu)}")
print(f"Number of duplicate examples in df_ps_jw_website: {len(duplicates_ps_jw)}")
print(f"Number of duplicate examples in df_trab_warao_completo: {len(duplicates_trab)}")
print(f"Number of duplicate examples in df_bible_verses: {len(duplicates_bible)}")

Number of duplicate examples in df_curso_basico_expanded: 36
Number of duplicate examples in df_warao_aribu_guide: 10
Number of duplicate examples in df_ps_jw_website: 0
Number of duplicate examples in df_trab_warao_completo: 0
Number of duplicate examples in df_bible_verses: 11


In [29]:
df_all = pd.concat([df_curso_basico_expanded, df_ps_jw_website, df_warao_aribu_guide, df_trab_warao_completo[['spanish_sentence', 'warao_sentence']], df_bible_verses])
# leave only spanish_sentence and warao_sentence columns
df_all = df_all[['warao_sentence', 'spanish_sentence']]
print(f"Total number of examples: {len(df_all)}")


Total number of examples: 9177


In [30]:
# extract rows that are exact duplicates
duplicates = df_all[df_all.duplicated(keep=False)]



print(f"Total number of duplicate examples: {len(duplicates)}")

display(duplicates.head(20))

Total number of duplicate examples: 57


,warao_sentence,spanish_sentence
3,Iji buratana najoroya,Tú comes cambur
13,Iji buratana najoroya,Tú comes cambur
321,Tai najorote,ella comerá.
322,Tai najorote,Él comerá.
323,Tai najorote,ella va a comer.
324,Tai najorote,Él va a comer.
353,Oko mite,Nosotros vamos a ver.
354,Oko mite,Nosotros vamos a ver.
468,Ine tamatika ja,Yo estoy aquí.
469,Nobotomo beoro tamate konae,Los niños trajeron al perro acá.


In [31]:
df_all.drop_duplicates(inplace=True)
print(f"Total number of examples after dropping duplicates: {len(df_all)}")


Total number of examples after dropping duplicates: 9148


In [32]:
output_filename = "parallel_data_all.csv"
output_path = os.path.join("output", output_filename)
df_all.to_csv(output_path, index=False)